In [179]:
import baseline
import importlib
importlib.reload(baseline)
from baseline import *
import numpy as np 
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [165]:
#Data loading and reduce memory usage by changing dtypes
train = reduce_mem_usage(pd.read_csv('train.csv'))
test = reduce_mem_usage(pd.read_csv('test.csv'))
shops = reduce_mem_usage(pd.read_csv('shops.csv'))
items = reduce_mem_usage(pd.read_csv('items.csv'))
categories = reduce_mem_usage(pd.read_csv('item_categories.csv'))

Mem. usage decreased to 14.48 Mb (80.0% reduction)
Mem. usage decreased to  1.43 Mb (70.8% reduction)
Mem. usage decreased to  0.00 Mb (40.9% reduction)
Mem. usage decreased to  0.27 Mb (59.4% reduction)
Mem. usage decreased to  0.00 Mb (42.4% reduction)


In [170]:
#Check if our train data is valid
column_types = {'date_block_num': 'int8', 'shop_id': 'int8', 'item_id': 'int16', 'item_price': 'float16', 'item_cnt_day': 'float16'}
values_ranges = {'date_block_num': (0, 33), 'shop_id': (0, 59), 'item_id': (0, 22169), 'item_price': (0.07, 42980.0), 'item_cnt_day': (0, 669)}
Validator(column_types = column_types, value_ranges = values_ranges, check_missing = True, check_duplicates=True).fit_transform(train)

'Data is valid'

In [171]:
#Target - item_cnt_month
target_group = (train.groupby(['date_block_num', 'shop_id', 'item_id'])['item_cnt_day']
                .sum().rename('item_cnt_month').reset_index())
#From EDA step we do not see linear dependency between item_cnt_month and item_price.
#Feature 'revenue' will give us more imformation about target
train['revenue'] = train['item_price'] * train['item_cnt_day']

In [172]:
'''
According to EDA: we do have a lot of data without full range during analysing period
and also our test set contains shop_id&item_id pairs that are nor presented in train set at all
'''
columns = ['date_block_num', 'shop_id', 'item_cnt_month']
full_data = full_data_creation(df = train, agg_group = columns, periods = train.date_block_num.nunique())

In [173]:
#Merge between full data and train set
full_data = full_data.merge(target_group, on = columns, how = 'left')
#test set concatenation with full_data
test['date_block_num'] = 34
del test['ID']
full_data = pd.concat([full_data, test], keys = columns, ignore_index=True, sort = False)

C:\Users\User\AppData\Local\Temp\ipykernel_71660\320426206.py:6: FutureWarning: The behavior of pd.concat with len(keys) != len(objs) is deprecated. In a future version this will raise instead of truncating to the smaller of the two sequences
  full_data = pd.concat([full_data, test], keys = columns, ignore_index=True, sort = False)


In [174]:
'''
We need:
1. fill all missing values as item&shop pairs + test set have been added
2. clip our target variable - original condition
'''
full_data = full_data.fillna(0)
full_data['item_cnt_month'] = full_data['item_cnt_month'].clip(0,20)

In [176]:
#Merge with other datasets
full_data = full_data.merge(shops, on = 'shop_id', how = 'left')
full_data = full_data.merge(items, on = 'item_id', how = 'left')
full_data = full_data.merge(categories, on = 'item_category_id', how = 'left')

In [182]:
#Columns we are planning to work
Work_columns = ['date_block_num', 'shop_id', 'item_cnt_month', 'item_id', 'city_id', 'item_category_id', 'main_category_id','minor_category_id']
full_data = full_data.loc[:, Work_columns]

#As we make transformations during DQL with shop_id, we will encode it with LabelEncoding
full_data['shop_id'] = LabelEncoder().fit_transform(full_data['shop_id'])